In [42]:
import pandas as pd
from datetime import timedelta, date
import geopandas as gpd
from uuid import uuid4

In [45]:

from datetime import datetime, timedelta
import pandas as pd
from shapely.geometry import Point
import geopandas as gpd
#import contextily as ctx
import matplotlib.pyplot as plt
datelist = pd.date_range(datetime.today(), periods=100).tolist()
from time import sleep
import dask.dataframe as dd
from dateutil.parser import parse
from dateutil.relativedelta import relativedelta
import os
import glob

from shapely.geometry import Polygon, MultiPolygon, LineString, GeometryCollection

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import geopandas as gpd



## Load heatwaves and get initial control dates

In [48]:
#heatwaves = pd.read_csv(f'/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/heatwave/heatwaves_90.csv')
heatwaves = pd.read_csv('THI_China_90_2023.csv')

In [50]:
heatwaves['month'].unique()

array([6, 7, 8, 5, 9])

In [52]:
heatwaves['datetime'] = pd.to_datetime(heatwaves['datetime'])

In [56]:
heatwaves['control_day'] = heatwaves['datetime'] - timedelta(days=7)

In [58]:
heatwaves['day_of_week'] = [x.weekday() for x in heatwaves['datetime']]

In [61]:
heatwaves_active = heatwaves[heatwaves['heatwave'] == 1]
heat_days = heatwaves_active.groupby('location')['datetime'].unique().reset_index().set_index('location')

In [63]:
heat_days = heat_days.to_dict()['datetime']

In [67]:
heatwaves_active.reset_index(inplace=True)

In [69]:
heatwaves_active

,index,Unnamed: 0.1,Unnamed: 0,datetime,number,spatial_ref,t2m,d2m,rh,thi,...,year,location,datetime_ord,pct,level_0,heatwave,heat_ID,Heat Wave Duration,control_day,day_of_week
0,0,30,674,2023-06-15,0.0,0.0,35.389908,14.821653,69.065350,34.094592,...,2023,"beijingshi, beijingshi",738686,0.937255,30.0,1.0,e5e2f459-fb8f-437c-a1e9-ee67e08ce703,3.0,2023-06-08,3
1,1,31,675,2023-06-16,0.0,0.0,36.591713,15.919735,69.757706,34.783292,...,2023,"beijingshi, beijingshi",738687,0.956863,31.0,1.0,e5e2f459-fb8f-437c-a1e9-ee67e08ce703,NaN,2023-06-09,4
2,2,32,676,2023-06-17,0.0,0.0,35.361340,15.996984,61.345226,34.180206,...,2023,"beijingshi, beijingshi",738688,0.939869,32.0,1.0,e5e2f459-fb8f-437c-a1e9-ee67e08ce703,NaN,2023-06-10,5
3,3,33,680,2023-06-21,0.0,0.0,35.432350,13.088347,65.301544,33.095773,...,2023,"beijingshi, beijingshi",738692,0.901961,33.0,1.0,ddf2de40-8322-4423-b5ef-10c11377677f,4.0,2023-06-14,2
4,4,34,681,2023-06-22,0.0,0.0,37.430340,13.256146,59.699830,34.996177,...,2023,"beijingshi, beijingshi",738693,0.967320,34.0,1.0,ddf2de40-8322-4423-b5ef-10c11377677f,NaN,2023-06-15,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5433,5433,24173,294306,2023-07-27,0.0,0.0,33.306550,26.585602,80.712320,41.806721,...,2023,"aomentebiexingzhengqu, aomentebiexingzhengqu",738728,0.954248,56.0,1.0,539d49f5-d2ee-4c90-837c-50b8e3f3d5bb,NaN,2023-07-20,3
5434,5434,24174,294307,2023-07-28,0.0,0.0,33.363190,28.224762,88.044730,42.778965,...,2023,"aomentebiexingzhengqu, aomentebiexingzhengqu",738729,0.972549,57.0,1.0,539d49f5-d2ee-4c90-837c-50b8e3f3d5bb,NaN,2023-07-21,4
5435,5435,24175,294308,2023-07-29,0.0,0.0,30.235504,28.246490,89.696976,41.351937,...,2023,"aomentebiexingzhengqu, aomentebiexingzhengqu",738730,0.943791,58.0,1.0,539d49f5-d2ee-4c90-837c-50b8e3f3d5bb,NaN,2023-07-22,5
5436,5436,24176,294326,2023-08-16,0.0,0.0,31.168365,27.726960,91.614360,42.177700,...,2023,"aomentebiexingzhengqu, aomentebiexingzhengqu",738748,0.959477,59.0,1.0,3c78b8ef-9921-423b-8d05-f7ba854fadb5,2.0,2023-08-09,2


In [72]:
full_df = pd.read_csv('daily_max_THI.csv')

full_df.rename(columns={'time': 'datetime'}, inplace=True)
full_df['datetime'] = pd.to_datetime(full_df['datetime'])
full_df['day'] = pd.DatetimeIndex(full_df['datetime']).day
full_df['month'] = pd.DatetimeIndex(full_df['datetime']).month
full_df['year'] = pd.DatetimeIndex(full_df['datetime']).year
full_df = full_df[~full_df['month'].isin([4, 10])]
full_df['location'] = full_df['adm2'] + ', ' + full_df['adm1']
full_df['datetime_ord'] = [date(x,y,z).toordinal() for x,y,z in zip(full_df['year'], full_df['month'], full_df['day'])]


In [74]:
full_df['temp_pct'] = full_df.groupby('location')['thi'].rank(pct=True)

In [76]:
heatwaves_active['num_iters'] = 'test'

In [78]:
# Updated control day search algorithm with less index searching -- faster for large datasets

for index, row in heatwaves_active.iterrows():
    subset = full_df[full_df['location'] == row['location']]
    control_day = heatwaves_active.iloc[index, -3]
    i = 0
    try:
        while ((subset[subset['datetime'] == control_day]['temp_pct'].values[0]>= 0.8) | 
               (subset[(subset['datetime'] == control_day)]['temp_pct'].values[0]<= 0.2)):


            heatwaves_active.iloc[index, -3] = heatwaves_active.iloc[index, -3] - timedelta(days=7)
            control_day = heatwaves_active.iloc[index, -3]

    except:
        heatwaves_active.iloc[index, -3] = heatwaves_active.iloc[index, -3] + timedelta(days=7)
        control_day = heatwaves_active.iloc[index, -3]

        while ((subset[subset['datetime'] == control_day]['temp_pct'].values[0]>= 0.8) | 
               (subset[(subset['datetime'] == control_day)]['temp_pct'].values[0]<= 0.2)):
        
            heatwaves_active.iloc[index, -3] = heatwaves_active.iloc[index, -3] + timedelta(days=7)
            control_day = heatwaves_active.iloc[index, -3]
            
    print(index / len(heatwaves_active))

0.0
0.00018389113644722325
0.0003677822728944465
0.0005516734093416697
0.000735564545788893
0.0009194556822361162
0.0011033468186833395
0.0012872379551305626
0.001471129091577786
0.0016550202280250091
0.0018389113644722325
0.002022802500919456
0.002206693637366679
0.002390584773813902
0.0025744759102611253
0.002758367046708349
0.002942258183155572
0.003126149319602795
0.0033100404560500183
0.003493931592497242
0.003677822728944465
0.003861713865391688
0.004045605001838912
0.004229496138286134
0.004413387274733358
0.004597278411180581
0.004781169547627804
0.004965060684075028
0.0051489518205222505
0.005332842956969474
0.005516734093416698
0.00570062522986392
0.005884516366311144
0.006068407502758367
0.00625229863920559
0.006436189775652814
0.0066200809121000365
0.00680397204854726
0.006987863184994484
0.007171754321441706
0.00735564545788893
0.007539536594336153
0.007723427730783376
0.0079073188672306
0.008091210003677823
0.008275101140125045
0.008458992276572269
0.008642883413019492
0.

In [32]:
heatwaves_active.to_csv('THI_heatwave_control_90.csv')